In [9]:
import requests
from bs4 import BeautifulSoup
from akshare.utils import demjson
import pandas as pd
from datetime import datetime, timedelta
from akshare.tool.trade_date_hist import tool_trade_date_hist_sina

# ===== 1. 数据预处理 ===== 
import numpy as np
import torch
import torch.nn as nn
from torch  import amp
from torch.utils.data  import Dataset, DataLoader
from sklearn.preprocessing  import MinMaxScaler
import joblib
import os
from tqdm.notebook import tqdm
# import py_mini_racer
import time
import talib

In [10]:
# def is_trading_day():
#     today = datetime.now().strftime('%Y-%m-%d') 
#     # 获取交易日历（新浪财经）
#     trade_date_df = tool_trade_date_hist_sina() 
#     # 判断今天是否在交易日历中
#     return today in trade_date_df['trade_date'].values

# if is_trading_day():
#     print("今天是交易日")
# else:
#     print("今天不是交易日")

# is_trading_day = is_trading_day()
# today = datetime.now().date()
trade_date_df = tool_trade_date_hist_sina() 
 
# 确保日期列是字符串格式
trade_date_list = trade_date_df['trade_date'].astype(str).tolist()
 
# 获取当前日期
today = datetime.now().date() - timedelta(days=1)
lastday = today - timedelta(days=1)
# 找到最近交易日
recent_trade_date = today 
while str(recent_trade_date) not in trade_date_list:
    recent_trade_date -= timedelta(days=1)
    lastday -= timedelta(days=1)
while str(lastday) not in trade_date_list:
    lastday -= timedelta(days=1)

print(f"最近交易日：{recent_trade_date}")
print(f"上一个交易日：{lastday}")

最近交易日：2025-11-14
上一个交易日：2025-11-13


In [11]:
start_year = "2025"
current_year = datetime.now().year

# js_code = py_mini_racer.MiniRacer()
# with open('ths.js', 'r', encoding="utf-8") as f:
#     js_content = f.read()
# js_code.eval(js_content)
# v_code = js_code.call("v")

headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/89.0.4389.90 Safari/537.36",
        # "Cookie": f"v={v_code}",
    }
headers_bk = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/89.0.4389.90 Safari/537.36",
            "Referer": "https://q.10jqka.com.cn",
            "Host": "d.10jqka.com.cn",
            # "Cookie": f"v={v_code}",
        }

gn_link_code_ls = []
thshy_link_code_ls = []
bk_data_ls = []
code_ls = []

for suffix in ["gn/","thshy/"]:
    url = f"https://q.10jqka.com.cn/{suffix}"
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, features="lxml")
    divs = soup.find_all(name="div", attrs={"class": "cate_items"})
    for div in divs:
        # 在每个div中查找所有的a标签
        links = div.find_all('a') 
        for link in links:
            # 获取href属性
            href = link.get('href') 
            # 获取文本内容
            text = link.text
            if suffix == "thshy/":
                parts = href.rstrip('/').split('/')
                code = parts[-1]
                thshy_link_code_ls.append((code,  text))
            else:
                gn_link_code_ls.append((href,  text))


In [12]:
print(len(gn_link_code_ls))
count = 0
for symbol_url, name in gn_link_code_ls:
    try:
        r = requests.get(symbol_url, headers=headers)
        soup_today = BeautifulSoup(r.text, features="lxml")
        symbol_code = soup_today.find(name="div", attrs={"class": "board-hq"}).find("span").text
        big_df = pd.DataFrame()
        temp_start_year = start_year
        if os.path.exists(f"./data/{symbol_code}.csv"):
            big_df = pd.read_csv(f"./data/{symbol_code}.csv",  encoding="utf-8")
            if not big_df.empty:
                big_df.iloc[:, 0] = pd.to_datetime(big_df.iloc[:, 0], errors="coerce").dt.date
                if big_df.iloc[-1, 0] >= recent_trade_date:
                    bk_data_ls.append((name, symbol_code, big_df))
                    code_ls.append(symbol_code)
                    continue
                elif big_df.iloc[-1, 0].year >= int(temp_start_year):
                    temp_start_year = str(big_df.iloc[-1, 0].year)
                    year_mask = big_df.iloc[:,0].apply(lambda x: x.year)  ==  int(temp_start_year) 
                    if year_mask.any():
                        first_index = year_mask.idxmax()
                    else:
                        first_index = None
                    if first_index is not None:
                        big_df = big_df.iloc[:first_index]
                elif (big_df.iloc[-1, 0] == lastday):
                    dd_texts = [dd.text for dd in soup_today.find_all("dd")] 
                    close = soup_today.select_one('span.board-xj.arr-rise,  span.board-xj.arr-fall').text
                    open, low, high, volume, amount = dd_texts[0], dd_texts[2], dd_texts[3], dd_texts[4], dd_texts[9]
                    new_list = [
                        recent_trade_date,
                        float(open),
                        float(high),
                        float(low),
                        float(close),
                        float(volume.replace(",", "")),
                        float(amount.replace(",", "")),
                    ]
                    new_row = pd.DataFrame([new_list], columns=big_df.columns) 
                    big_df = pd.concat([big_df,  new_row], ignore_index=True)
                    bk_data_ls.append((name, symbol_code, big_df))
                    code_ls.append(symbol_code)
                    big_df.to_csv(f"./data/{symbol_code}.csv", index=False, encoding='utf-8-sig')
                    continue
        
        headers_d = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/89.0.4389.90 Safari/537.36",
            "Referer": symbol_url,
            "Host": "d.10jqka.com.cn"
        }
        time.sleep(1)
        for year in tqdm(range(int(temp_start_year), current_year + 1), leave=False):
            url = f"http://d.10jqka.com.cn/v4/line/bk_{symbol_code}/01/{year}.js"
            r = requests.get(url, headers=headers_d)
            data_text = r.text
            if r.status_code != 200:
                print("not 200!", r.status_code, symbol_url, url)
                continue
            try:
                temp_df = demjson.decode(data_text[data_text.find("{") : -1])
                temp_df = pd.DataFrame(temp_df["data"].split(";"))
                temp_df = temp_df.iloc[:, 0].str.split(",", expand=True).iloc[:,  :7]
                temp_df.columns  = ["date", "open", "high", "low", "close", "volume", "amount"]
                big_df = pd.concat(objs=[big_df, temp_df], ignore_index=True)
            except Exception as e:  # noqa: E722
                print(e, "decode fail:",count,"  ", url, "text as:", data_text)
                break
        big_df.columns  = ["date", "open", "high", "low", "close", "volume", "amount"]
        big_df["date"] = pd.to_datetime(big_df["date"], errors="coerce").dt.date
        big_df["open"] = pd.to_numeric(big_df["open"], errors="coerce")
        big_df["high"] = pd.to_numeric(big_df["high"], errors="coerce")
        big_df["low"] = pd.to_numeric(big_df["low"], errors="coerce")
        big_df["close"] = pd.to_numeric(big_df["close"], errors="coerce")
        big_df["volume"] = pd.to_numeric(big_df["volume"], errors="coerce")
        big_df["amount"] = pd.to_numeric(big_df["amount"], errors="coerce")
        
        if big_df.iloc[-1].isna().any(): 
            big_df = big_df.iloc[:-1]

        if big_df["date"].iloc[-1] == lastday:
            dd_texts = [dd.text for dd in soup_today.find_all("dd")] 
            close = soup_today.select_one('span.board-xj.arr-rise,  span.board-xj.arr-fall').text
            open, low, high, volume, amount = dd_texts[0], dd_texts[2], dd_texts[3], dd_texts[4], dd_texts[9]
            new_list = [
                recent_trade_date,
                float(open),
                float(high),
                float(low),
                float(close),
                float(volume.replace(",", "")),
                float(amount.replace(",", "")),
            ]
            new_row = pd.DataFrame([new_list], columns=big_df.columns) 
            big_df = pd.concat([big_df,  new_row], ignore_index=True)
        bk_data_ls.append((name, symbol_code, big_df))
        code_ls.append(symbol_code)
        big_df.to_csv(f"./data/{symbol_code}.csv", index=False, encoding='utf-8-sig')
        count+=1
    except Exception as e:
        print(e, " wrong data:",count,"  ", f"https://d.10jqka.com.cn/v4/line/bk_{symbol_code}/01/{year}.js", symbol_url, big_df.tail())
        break


352


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [29]:
# # http://q.10jqka.com.cn/thshy/detail/code/881138/
# headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
#         "Chrome/89.0.4389.90 Safari/537.36",
#         # "Cookie": f"v={v_code}",
#     }
# headers_bk = {
#             "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
#             "Chrome/89.0.4389.90 Safari/537.36",
#             "Referer": "https://q.10jqka.com.cn",
#             "Host": "d.10jqka.com.cn",
#             # "Cookie": f"v={v_code}",
#         }
# url =  "https://d.10jqka.com.cn/v4/line/bk_886056/01/2025.js"
# print(url)
# r = requests.get(url, headers=headers)
# print(r.status_code)
# print(r.text)
# symbol_url = "https://q.10jqka.com.cn/gn/detail/code/308614/"
# print(symbol_url)
# r = requests.get(symbol_url, headers=headers)
# print(r.status_code)
# print(r.text)
# # for name, symbol_code, big_df in bk_data_ls:
# #     print(symbol_code, big_df.shape)

In [14]:

print(len(thshy_link_code_ls))

for symbol_code, name in thshy_link_code_ls:
    big_df = pd.DataFrame()
    if os.path.exists(f"./data/{symbol_code}.csv"):
        big_df = pd.read_csv(f"./data/{symbol_code}.csv",  encoding="utf-8")
        if not big_df.empty:
            big_df.iloc[:, 0] = pd.to_datetime(big_df.iloc[:, 0], errors="coerce").dt.date
            if big_df.iloc[-1, 0] >= recent_trade_date:
                bk_data_ls.append((name, symbol_code, big_df))
                code_ls.append(symbol_code)
                continue
            elif big_df.iloc[-1, 0].year >= int(temp_start_year):
                temp_start_year = str(big_df.iloc[-1, 0].year)
                year_mask = big_df.iloc[:,0].apply(lambda x: x.year)  ==  int(temp_start_year) 
                if year_mask.any():
                    first_index = year_mask.idxmax()
                else:
                    first_index = None
                if first_index is not None:
                    big_df = big_df.iloc[:first_index]

    try:
        for year in tqdm(range(int(temp_start_year), current_year + 1), leave=False):
            url = f"http://d.10jqka.com.cn/v4/line/bk_{symbol_code}/01/{year}.js"
            r = requests.get(url, headers=headers_bk)
            data_text = r.text
            try:
                temp_df = demjson.decode(data_text[data_text.find("{") : -1])
                temp_df = pd.DataFrame(temp_df["data"].split(";"))
                temp_df = temp_df.iloc[:, 0].str.split(",", expand=True).iloc[:,  :7]
                temp_df.columns  = ["date", "open", "high", "low", "close", "volume", "amount"]
                big_df = pd.concat(objs=[big_df, temp_df], ignore_index=True)
            except Exception as e:  # noqa: E722
                print(e, "decode fail:", url, "text as:", data_text)
                break
        big_df = big_df.iloc[:,  :7]
        big_df.columns  = ["date", "open", "high", "low", "close", "volume", "amount"]
        big_df["date"] = pd.to_datetime(big_df["date"], errors="coerce").dt.date
        big_df["open"] = pd.to_numeric(big_df["open"], errors="coerce")
        big_df["high"] = pd.to_numeric(big_df["high"], errors="coerce")
        big_df["low"] = pd.to_numeric(big_df["low"], errors="coerce")
        big_df["close"] = pd.to_numeric(big_df["close"], errors="coerce")
        big_df["volume"] = pd.to_numeric(big_df["volume"], errors="coerce")
        big_df["amount"] = pd.to_numeric(big_df["amount"], errors="coerce")

        if big_df.iloc[-1].isna().any(): 
            big_df = big_df.iloc[:-1]

        if big_df["date"].iloc[-1] == lastday:
            symbol_url = f"https://q.10jqka.com.cn/thshy/detail/code/{symbol_code}/"
            r = requests.get(symbol_url, headers=headers)
            soup_today = BeautifulSoup(r.text, features="lxml")
            dd_texts = [dd.text for dd in soup_today.find_all("dd")] 
            close = soup_today.select_one('span.board-xj.arr-rise,  span.board-xj.arr-fall').text
            open, low, high, volume, amount = dd_texts[0], dd_texts[2], dd_texts[3], dd_texts[4], dd_texts[9]
            new_list = [
                recent_trade_date,
                float(open),
                float(high),
                float(low),
                float(close),
                float(volume.replace(",", "")),
                float(amount.replace(",", "")),
            ]
            new_row = pd.DataFrame([new_list], columns=big_df.columns) 
            big_df = pd.concat([big_df,  new_row], ignore_index=True)
       
        bk_data_ls.append((name, symbol_code, big_df))
        code_ls.append(symbol_code)
        big_df.to_csv(f"./data/{symbol_code}.csv", index=False, encoding='utf-8-sig')
    except Exception as e:
        print(e, " wrong data:", f"https://d.10jqka.com.cn/v4/line/bk_{symbol_code}/01/{year}.js", symbol_url, big_df.tail())
        break


90


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
print(bk_data_ls[0][0])
print(bk_data_ls[0][1])
print(bk_data_ls[0][2].head())
print(bk_data_ls[0][2].tail())

阿尔茨海默概念
886056
         date     open     high      low    close     volume        amount
0  2025-01-02  989.548  999.124  968.150  974.704  512509970  6.680677e+09
1  2025-01-03  974.714  986.675  962.398  965.056  475802500  6.222582e+09
2  2025-01-06  971.721  987.276  960.495  979.601  525790230  6.659436e+09
3  2025-01-07  979.367  979.367  951.360  969.033  966740090  1.109609e+10
4  2025-01-08  967.908  974.077  945.650  965.447  771668230  9.432990e+09
           date      open      high       low     close     volume  \
205  2025-11-10  1298.777  1313.862  1298.777  1313.439  627143680   
206  2025-11-11  1313.721  1318.361  1306.688  1315.799  526188270   
207  2025-11-12  1317.354  1328.205  1317.019  1325.098  622175250   
208  2025-11-13  1323.767  1333.820  1312.062  1333.805  639962250   
209  2025-11-14  1326.357  1351.269  1325.553  1339.382  763120370   

           amount  
205  9.625723e+09  
206  7.555462e+09  
207  9.793342e+09  
208  1.072351e+10  
209  1.256780e

In [17]:
range_len = 50

In [23]:
def check_parameter(bk_data_ls):
    count = 0
    for name, symbol_code, stock_zh_df in bk_data_ls:
        open = stock_zh_df["open"].tail(range_len)
        close = stock_zh_df["close"].tail(range_len)
        high = stock_zh_df["high"].tail(range_len)
        low = stock_zh_df["low"].tail(range_len)
        volume = stock_zh_df["volume"].tail(range_len)

        ma5 = talib.EMA(close, timeperiod=5)
        ma10 = talib.EMA(close, timeperiod=10)
        vol_ma5 = talib.EMA(volume, timeperiod=5)
        atr = talib.NATR(high, low, close, timeperiod=8) #波动
        atr_ma = talib.EMA(atr, timeperiod=5)
        macd, macdsignal, macdhist = talib.MACD(close, fastperiod=7, slowperiod=18, signalperiod=6)

        rsi = talib.RSI(close, timeperiod=14)  # RSI相对强弱指标
        cci = talib.CCI(high, low, close, timeperiod=20)  # 顺势指标
        # money_flow = (2*close - low - high) / (high - low) * volume  # 简易资金流
        money_flow = talib.MFI(high, low, close, volume, timeperiod=9)
        # money_flow_max= money_flow.shift(3).rolling(4).max().iloc[-1]
        today_candlestick = abs(close.iloc[-3] - open.iloc[-3])
        yesterday_candlestick = abs(close.iloc[-4] - open.iloc[-4])
        
        if (close.iloc[-3] > ma5.iloc[-3] and (ma5.iloc[-3] > ma10.iloc[-3]  or (ma5.iloc[-3] > ma5.iloc[-4] > ma5.iloc[-5]))) and \
            rsi.iloc[-3] < 60 and atr.iloc[-3]  > atr_ma.iloc[-3] and open.iloc[-3]  < close.iloc[-3]  * 1.05 and \
                volume.iloc[-3] >= vol_ma5.iloc[-3] * 1.2  and \
                (macd.iloc[-3] > macdsignal.iloc[-3] and macdhist.iloc[-3] > abs(macdhist.iloc[-4])):
            if close.iloc[-1] > close.iloc[-2]:
                print(f"""name: {name}, symbol_code: {symbol_code} result: 趋势启动+量价齐升型 \n """)
            else:
                print(f"""XXXXXXname: {name}, symbol_code: {symbol_code} wrong result: 趋势启动+量价齐升型 \n """)
                count += 1
        elif (rsi.iloc[-3] < 40) and (cci.iloc[-3] < -80) and \
            today_candlestick > 0.3 * yesterday_candlestick and \
            money_flow.iloc[-3] > money_flow.iloc[-4]:
            if close.iloc[-1] > close.iloc[-2]:
                print(f"""name: {name}, symbol_code: {symbol_code} result: 超跌反转+资金异动型 \n """)
            else:
                print(f"""XXXXXXname: {name}, symbol_code: {symbol_code} wrong result: 超跌反转+资金异动型 \n """)
                count += 1
    print("wrong count", count)

check_parameter(bk_data_ls)

XXXXXXname: 国资云, symbol_code: 885977 wrong result: 超跌反转+资金异动型 
 
XXXXXXname: 数字货币, symbol_code: 885866 wrong result: 超跌反转+资金异动型 
 
wrong count 2


In [24]:
def run_analysis(bk_data_ls):
    res_ls = []
    for name, symbol_code, stock_zh_df in bk_data_ls:
        open = stock_zh_df["open"].tail(range_len)
        close = stock_zh_df["close"].tail(range_len)
        high = stock_zh_df["high"].tail(range_len)
        low = stock_zh_df["low"].tail(range_len)
        volume = stock_zh_df["volume"].tail(range_len)

        ma5 = talib.EMA(close, timeperiod=5)
        ma10 = talib.EMA(close, timeperiod=10)
        vol_ma5 = talib.EMA(volume, timeperiod=5)
        atr = talib.NATR(high, low, close, timeperiod=8)
        atr_ma = talib.EMA(atr, timeperiod=5)
        macd, macdsignal, macdhist = talib.MACD(close, fastperiod=7, slowperiod=18, signalperiod=6)#快速EMA：6~8，慢速EMA：15~20，信号线：5~7

        rsi = talib.RSI(close, timeperiod=14)  # RSI相对强弱指标
        cci = talib.CCI(high, low, close, timeperiod=20)  # 顺势指标
        # money_flow = (2*close - low - high) / (high - low) * volume  # 简易资金流
        money_flow = talib.MFI(high, low, close, volume, timeperiod=9)
        # money_flow_max= money_flow.shift(1).rolling(4).max().iloc[-1]
        today_candlestick = abs(close.iloc[-1] - open.iloc[-1])
        yesterday_candlestick = abs(close.iloc[-2] - open.iloc[-2])

        if close.iloc[-1] > ma5.iloc[-1] and (ma5.iloc[-1] > ma10.iloc[-1] or (ma5.iloc[-1] > ma5.iloc[-2] > ma5.iloc[-3])) and \
                rsi.iloc[-1] < 60 and atr.iloc[-1]  > atr_ma.iloc[-1] and open.iloc[-1]  < close.iloc[-1]  * 1.05 and \
                volume.iloc[-1] >= vol_ma5.iloc[-1] * 1.2 and \
                macd.iloc[-1] > macdsignal.iloc[-1] and macdhist.iloc[-1] > abs(macdhist.iloc[-2]):
            res_ls.append(f"""name: {name}, symbol_code: {symbol_code} result: 趋势启动+量价齐升型 \n """)
        elif (rsi.iloc[-1] < 40) and (cci.iloc[-1] < -80) and \
            today_candlestick > 0.3 * yesterday_candlestick and \
            money_flow.iloc[-1] > money_flow.iloc[-2]:
            res_ls.append(f"""name: {name}, symbol_code: {symbol_code} result: 超跌反转+资金异动型 \n """)

    # for result in res_ls:
    print("hi")
    print(res_ls)
    # with open('today_suggestions.txt', 'a', encoding='utf-8') as today_suggestions,open('history_suggestions.txt', 'a', encoding='utf-8') as history_suggestions:
    #     for result in res_ls:
    #         today_suggestions.write(result)
    #         history_suggestions.write(result)


run_analysis(bk_data_ls)

hi
['name: 无人零售, symbol_code: 885768 result: 趋势启动+量价齐升型 \n ']
